In [1]:
#string='PAGO A PROEX -BRASIL PRÉSTAMO CFA009759 VCTO. 08-09-2022 POR CUENTA DE TGN , NTI. 016483 VALOR 08-09-2022 CAPITAL USD 91.133,09 INTERESES EUR 363.608,09'
#string='PAGO A AFD PRÉSTAMO CBO-1006-01-F VCTO. 10-08-2022 POR CUENTA DE ENDE , VALOR 10-08-2022 CAPITAL EUR 2.068.965,52 INTERESES EUR 187.282,99'
string='PAGO A BID PRÉSTAMO 2460/BL-BO VCTO. 19-07-2022 POR CUENTA DE GOB.AUT.DEPT.CHUQUIS , VALOR 19-07-2022 CAPITAL USD 206.328,00 INTERESES USD 159.306,06'
#string='TRANSFERENCIA RECIBIDA DEL EXTERIOR SEGÚN MENSAJES SWIFT Nos. 12555-12554 (REM.EXT.) DE FECHA 28-07-2022 POR DESEMBOLSO DE CAF PRÉSTAMO CAF011750 APOYO PLAN DE VACUNACIÓN COVID 19    '
#string='PAGO A KFW ALEMANIA PRÉSTAMO KfW 200965608 VCTO. 30-06-2022 POR CUENTA DE TGN , NTI. 016217 VALOR 30-06-2022  INTERESES EUR 3.153,54 COMISIONES EUR 11.448,82     '
#string='VENTA DE DIVISAS CON TRANSFERENCIA DE FONDOS A SOLICITUD DE UNIVERSIDAD MAYOR DE SAN ANDRES SEGUN SOLICITUD 16099 REF: SOLICITUD DE VENTA DE DIVISAS AL EXTERIOR DEL PAGO AL DOC.EXT. JUAN FERNANDO CORDOBA MARENTES DOC.MODULO '
string="'TRANSFERENCIA'||RECIBIDA DEL EXTERIOR SEGÚN MENSAJES SWIFT NOS. 1135-1134 (REM.EXT.) DE FECHA 27-01-2022 POR DESEMBOLSO DE LA OPEP PRÉSTAMO 12601P WA NO.02.       "

arch='goideuda 2022.xls'
casosEspeciales=['BID','CAF','IDA']
menosEstas=['cve_debe_haber','cod_moneda','cod_movimiento','esdeuda','DesembPtmo','','']
import re


In [2]:
# JUNCIONES en modulo diferente oe
import os
from dbfread import DBF
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
def getExtension(archivo):
    extension = os.path.splitext(archivo)[1]
    return extension
def leer(nombreArchivo):
    if getExtension(nombreArchivo) == '.dbf':
        table = DBF(nombreArchivo, encoding='latin', load=True)
        return pd.DataFrame(iter(table))
    else:
        #para los que no son dbf
        return pd.read_excel(nombreArchivo)
def reducirColumnas (nombreArchivo,ddff):
        if getExtension (nombreArchivo) == '.dbf':
            bdReducida = ddff.loc[:, ('fecha_dia', 'cve_debe_h', 'monto_mo','factor_conv_mo_mn',
                                     'cod_moneda', 'cod_movimi', 'nom_movimi',
                                         'monto_mn', 'glosa_comp', 'nro_compro', 'cod_mayor')]
        else:

            bdReducida = ddff.loc[:, ('fecha_dia', 'cve_debe_haber', 'monto_mo','factor_conv_mo_mn',
                                     'cod_moneda', 'cod_movimiento', 'nom_movimiento',
                                         'monto_mn', 'glosa_comprob', 'nro_comprob', 'cod_mayor')]
        return bdReducida

In [3]:
def extraeMonto(texto,key):
    x = re.findall(key, texto)
    if x:
        xx=x[0][1].replace('.','').replace(',','.').replace(';','')
        try:
            float(xx)
        except:
            monto="algo"
        else:
            monto=float(xx)
        #y=[x[0][0],float(xx)]
        return monto

def extraeMoneda(texto,key):
    x = re.findall(key, texto)
    if x:
        try:
            x[0][0]
        except:
            moneda="---"
        else:
            moneda=x[0][0]
        return moneda
    
def extraeEntidades(texto,key):
    x = re.findall(key, texto)
    if x:
        if x[0]:
            return str(x[0])
        else:
            return "nd"

def extraeEntidadPtmoDesemb(texto,key,flag):
    x = re.findall(key, texto)
    if x:
        if x[0]:
            if flag==1:
                return str(x[0][1])
            else:
                return str(x[0][0])
        else:
            return "nd"



def extraePtmoDesemb(x):
    if x['DesembAcreedor'] in casosEspeciales:
        return extraeEntidadPtmoDesemb(x['glosa_comprob'],ptmoDesemb,1)
    else:
        return extraeEntidadPtmoDesemb(x['glosa_comprob'],ptmoDesemb,0)
    
def separa(s):
    letras='([A-Z])'
    numeros='(\d+)'
    x = re.findall(letras, s)
    y = re.findall(numeros, s)
    xx=''.join(x) if x else 'nuay'
    yy=int(y[0]) if y else 'nuay'
    return [xx,yy]

In [4]:
mayores = [10,381,404,406]
montos=['CAPITAL ','INTERESES ','COMISIONES ']
desAcree=['DESEMBOLSO DE ','(?= PRÉSTAMO)']
desPtmo=['POR DESEMBOLSO DE \w+ PRÉSTAMO ',' ']

akey=['PAGO A ','(?= PRÉSTAMO)']
pkey=['PRÉSTAMO ',' VCTO.']
dkey=['POR CUENTA DE ',' ,']

buscaMontos = '(\w+) ((\w+)($|\S+))'
buscaEntidades ='(.+?)'

buscaEntidades1 ='((.+?) (.+?)|(.+?))'

acreedorDesemb=desAcree[0]+buscaEntidades+desAcree[1]
ptmoDesemb=desPtmo[0]+buscaEntidades1+desPtmo[1]

acreedor=akey[0]+buscaEntidades+akey[1]
ptmo=pkey[0]+buscaEntidades+pkey[1]
deudor=dkey[0]+buscaEntidades+dkey[1]

In [5]:
bdBruto = leer(arch)
bdBruto.columns = map(str.lower, bdBruto.columns)
bdBruto=reducirColumnas(arch,bdBruto)        
print("bruta",bdBruto.shape)
print ("reudcida ", reducirColumnas(arch,bdBruto).shape)#
#bdBruto.columns
#bdBruto.head()

WARNING *** file size (64334084) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
bruta (39764, 11)
reudcida  (39764, 11)


In [6]:
def esdeuda(texto,criterio1,criterio2):
    if extraeEntidades(texto,criterio1) or extraeEntidades(texto,criterio2):
        return True
    else:
        return False

In [7]:
bdBruto["esdeuda"] = bdBruto["glosa_comprob"].map(lambda x:esdeuda(x,acreedor,acreedorDesemb))
bdDeuda=bdBruto[bdBruto["esdeuda"]==True]
bdDeuda= bdDeuda[bdDeuda["cod_mayor"].isin(mayores)]
#bdDeuda.head()
#bdDeuda.shape

In [8]:
def montoDesembolsado (x):
    if x['cve_debe_haber']=="D":
        #normalmente es Dólares, pero podrían haber otras monedas, hay que generalizar la fórmula oe.
        return round(x['monto_mn']/x['factor_conv_mo_mn'],2)
def montoPagadoK(x):
    if x['cve_debe_haber']=="H":
        try:
            x["PagoCapitalMO"]*x['factor_conv_mo_mn']
        except:
            return None
        else:    
            return round(x["PagoCapitalMO"]*x['factor_conv_mo_mn']/6.86,2)
        
def montoPagadoI(x):
    if x['cve_debe_haber']=="H":
        try:
            x["PagoInteresesMO"]*x['factor_conv_mo_mn']
        except:
            return None
        else:    
            return round(x["PagoInteresesMO"]*x['factor_conv_mo_mn']/6.86,2)   
def montoPagadoC(x):
    if x['cve_debe_haber']=="H":
        try:
            x["PagoComisionesMO"]*x['factor_conv_mo_mn']
        except:
            return None
        else:    
            return round(x["PagoComisionesMO"]*x['factor_conv_mo_mn']/6.86,2)   

In [9]:
       
bdDeuda["DesembAcreedor"]=bdBruto["glosa_comprob"].map(lambda x:extraeEntidades(x,acreedorDesemb))
bdDeuda["DesembPtmo"]=bdBruto["glosa_comprob"].map(lambda x:extraeEntidades(x,ptmoDesemb))
bdDeuda["DesembPtmoX"]=bdDeuda.apply(extraePtmoDesemb, axis=1)

bdDeuda["DesembEn$us"]=bdBruto.apply(montoDesembolsado, axis=1)

bdDeuda["ServicioAcreedor"]=bdBruto["glosa_comprob"].map(lambda x:extraeEntidades(x,acreedor))
bdDeuda["ServicioPtmo"]=bdBruto["glosa_comprob"].map(lambda x:extraeEntidades(x,ptmo))
bdDeuda["ServicioDeudor"]=bdBruto["glosa_comprob"].map(lambda x:extraeEntidades(x,deudor))
#moneda= extraeMontos(string,montos[2]+buscaMontos)[0]
#print(moneda)
bdDeuda["MonedaCapital"]=bdBruto["glosa_comprob"].map(lambda x:extraeMoneda(x,montos[0]+buscaMontos))
bdDeuda["PagoCapitalMO"]=bdBruto["glosa_comprob"].map(lambda x:extraeMonto(x,montos[0]+buscaMontos))
bdDeuda["PagoCapital$us"]=bdDeuda.apply(montoPagadoK, axis=1)

bdDeuda["MonedaIntereses"]=bdBruto["glosa_comprob"].map(lambda x:extraeMoneda(x,montos[1]+buscaMontos))
bdDeuda["PagoInteresesMO"]=bdBruto["glosa_comprob"].map(lambda x:extraeMonto(x,montos[1]+buscaMontos))
bdDeuda["PagoIntereses$us"]=bdDeuda.apply(montoPagadoI, axis=1)

bdDeuda["MonedaComisiones"]=bdBruto["glosa_comprob"].map(lambda x:extraeMoneda(x,montos[2]+buscaMontos))
bdDeuda["PagoComisionesMO"]=bdBruto["glosa_comprob"].map(lambda x:extraeMonto(x,montos[2]+buscaMontos))
bdDeuda["PagoComisiones$us"]=bdDeuda.apply(montoPagadoC, axis=1)

#bdDeuda.to_excel("borrar.xlsx")

#bdDeuda.head()



In [10]:
bdDeudaOK=bdDeuda.loc[:, ~bdDeuda.columns.isin(menosEstas)]
bdDeudaOK.to_excel("borrar1.xlsx", index=False)

In [11]:
print("este seria el acreedor del desembolso",extraeEntidades(string,acreedorDesemb))
print("este seria el ptmo del desembolso",extraePtmoDesemb(string,ptmoDesemb))
print("datos del prestamo del desembolso ",separa(str(extraeEntidades(string,ptmoDesemb))))
print("-----------------")
print("este seria el acreedor",extraeEntidades(string,acreedor))
print("este seria el prestamo",extraeEntidades(string,ptmo))
print("datos del prestamo",separa(str(extraeEntidades(string,ptmo))))
print("este seria el deudor",extraeEntidades(string,deudor))
print("-----------------")
print("este seria el capital",extraeMontos(string,monto[0]+buscaMontos))
print("este seria el interes",extraeMontos(string,monto[1]+buscaMontos))
print("este seria la comision",extraeMontos(string,monto[2]+buscaMontos))


este seria el acreedor del desembolso LA OPEP


TypeError: extraePtmoDesemb() takes 1 positional argument but 2 were given

In [ ]:

print("este seria el acreedor del desembolso",extraeEntidades(string,acreedorDesemb))
print("este seria el ptmo del desembolso",extraeEntidades(string,ptmoDesemb))
print("datos del prestamo del desembolso ",separa(str(extraeEntidades(string,ptmoDesemb))))

print("este seria el prestamo",extraeEntidades(string,ptmo))
print("datos del prestamo",separa(str(extraeEntidades(string,ptmo))))
print("este seria el acreedor",extraeEntidades(string,acreedor))
print("este seria el deudor",extraeEntidades(string,deudor))

print("este seria el capital",extraeMontos(string,montos[0]+buscaMontos))
print("este seria el interes",extraeMontos(string,montos[1]+buscaMontos))
print("este seria la comision",extraeMontos(string,montos[2]+buscaMontos))
